In [2]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime
import time
import random
from math import sqrt

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
file = '1minute_data_austin.csv'

In [8]:
data = pd.read_csv(file)

In [9]:
data

,dataid,localminute,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,...,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,wellpump1,winecooler1,leg1v,leg2v
0,661,2018-01-01 01:21:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,661,2018-01-01 01:00:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,661,2018-01-01 01:01:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,661,2018-01-01 01:02:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,661,2018-01-01 01:03:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13100534,7951,2018-02-07 15:26:00-06,0.0,0.007,NaN,NaN,NaN,-0.003,0.003,NaN,...,NaN,NaN,NaN,NaN,-0.001,NaN,NaN,NaN,122.726,122.830
13100535,7951,2018-02-07 15:27:00-06,0.0,0.008,NaN,NaN,NaN,-0.003,0.003,NaN,...,NaN,NaN,NaN,NaN,-0.001,NaN,NaN,NaN,122.800,122.738
13100536,7951,2018-02-07 15:28:00-06,0.0,0.005,NaN,NaN,NaN,-0.002,0.003,NaN,...,NaN,NaN,NaN,NaN,-0.001,NaN,NaN,NaN,122.945,122.728
13100537,7951,2018-02-07 15:29:00-06,0.0,0.004,NaN,NaN,NaN,-0.002,0.003,NaN,...,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,122.907,122.724


In [10]:
data.head()

,dataid,localminute,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,...,sprinkler1,sumppump1,utilityroom1,venthood1,waterheater1,waterheater2,wellpump1,winecooler1,leg1v,leg2v
0,661,2018-01-01 01:21:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,661,2018-01-01 01:00:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,661,2018-01-01 01:01:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,661,2018-01-01 01:02:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,661,2018-01-01 01:03:00-06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
ids = data.dataid.unique()
ids

array([ 661, 1642, 2335, 2818, 3039, 3456, 3538, 4031, 4373, 4767, 5746,
       6139, 7536, 7719, 7800, 7901, 7951, 8565, 9019, 9278, 8156, 8386,
       2361, 9922, 9160])

In [12]:
ids.shape

(25,)

In [15]:
#generate complex power using random power factor (between .9-.95)
def complex_power(real_power):
    power_factor = random.uniform(.9, .95)
    apparent_power = real_power / power_factor
    reactive_power = sqrt(apparent_power**2 - real_power**2)
    return str(round(real_power,6))+ '+' + str(round(reactive_power,6))+'j'

In [16]:
for id in ids:
    df = data[data['dataid']==id]
    
    # Fill NaN with mean of non-NaN value before and after it
    df['grid'] = (df['grid'].ffill()+df['grid'].bfill())/2
    df['grid'] = df['grid'].bfill().ffill()
    df = df.reset_index()
    
    #get first timestamp
    timestamp = str(df.iloc[0]['localminute'])
    if(timestamp[11]=='0'):
        timestamp = timestamp[:10] + ' ' + timestamp[12:-3] + ' CST'
    else:
        timestamp = timestamp[:-3] + ' CST'
    
    #create dataframe for player file output
    final_df = pd.DataFrame(index=range(df.shape[0]),columns=range(2))
    final_df.columns = ['time','grid']
    final_df['time'] = '+60s'
    final_df['grid'] = df['grid'] * 1000 #change to kW
    final_df['grid'] = final_df['grid'].map(complex_power) #convert to complex

    final_df['time'].iloc[0] = timestamp
    print(final_df.head())
    final_df.to_csv('./player_files/' + str(id) + '.player', header=None, index=None)

                     time                grid
0  2018-01-01 1:21:00 CST  1806.0+763.682476j
1                    +60s  1817.0+643.499627j
2                    +60s  1804.0+844.754468j
3                    +60s  1761.0+579.588339j
4                    +60s  1672.0+596.250673j
                     time               grid
0  2018-01-01 0:10:00 CST  359.0+142.608578j
1                    +60s  326.0+123.598073j
2                    +60s  325.0+155.813391j
3                    +60s  325.0+145.125706j
4                    +60s  445.0+185.221537j
                      time                grid
0  2018-01-01 18:36:00 CST  1565.0+704.550731j
1                     +60s  1489.0+666.207087j
2                     +60s  1482.0+574.246587j
3                     +60s  1477.0+525.443959j
4                     +60s  1477.0+543.460208j
                     time                grid
0  2018-01-01 0:00:00 CST  1373.0+542.505167j
1                    +60s  1287.0+541.962769j
2                    +60s  1300.0+